## ASSIGNMENT-2

Roll Numbers: 2023121013, 2023122002, 2023121006

Names: Himanshu Singh, Srinath Bhamidipati, Pavan Karke

### Instructions
 * Fill in the roll numbers and names of all the teams members in the cell above.
 * Code must be written in Python in Jupyter Notebooks. We highly recommend using anaconda distribution or at the minimum, virtual environments for this assignment.
 * All the code and result files should be uploaded in the github classroom.
 * You can use the in-built methods and **unless explicitly mentioned**, don't need to code from scratch for this assignment. Make sure your code is modular since you will be reusing them for future assignments.
 * All the representations are expected to be in a right-hand coordinate system. All the functions related to transformation matrices, quaternions, and 3D projection are expected to be coded by you.
 * You could split the Jupyter Notebook cells where TODO is written, but please try to avoid splitting/changing the structure of other cells.
 * All the visualization should be done inside the notebook unless specified otherwise.
 * Plagiarism will lead to heavy penalty.

## G2O Motion Model 

Using the following motion model, you have to first generate the "initialization" for all the poses/vertices using the "Given" information. Just like in the 1D case.
$$x_{k+1} = x_{k} + \Delta x_{(k,k+1)} \cos(\theta_k) - \Delta y_{(k,k+1)} \sin(\theta_k) \\
y_{k+1} = y_{k} + \Delta y_{(k,k+1)} \cos(\theta_k) + \Delta x_{(k,k+1)} \sin(\theta_k) \\
\theta_{k+1} = \theta_{k}+  \Delta \theta_{(k,k+1)}$$

Even the loop closure nodes are related by the above model, except that it need not necessarily be consecutive nodes k and k+1.

## PART 2

### 1. ICP Alignment with LieTorch (5 marks)
Resources: https://github.com/princeton-vl/lietorch/tree/master

Perform ICP Alignment of 2 PointClouds using LieTorch. Similar to what was expected in Part 1, you will be given a random Transformation matrix from the getTransform function. You are expected to transform the Toothless Point Cloud by this transformation, add noise to both sets of points (original and transformed) and estimate both, the Transformation matrix and the original point cloud.

Visualize your results using open3d

In [1]:
# Imports

import os

import lietorch
import numpy as np
import open3d as o3d
import torch

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def add_gaussian_noise(X, alpha=0.01):
    """ Adds Gaussian noise to a set of homogeneous points. """

    return X + torch.hstack([alpha * torch.randn(X.shape[0], 3), torch.zeros((X.shape[0], 1))])

def apply_transformation(X, T):
    """ Applies a 4x4 transformation matrix to a set of homogeneous points. """

    return X @ T.T

def euclidean_coordinates(X):
    """ Converts a set of points in homogeneous coordinate system to Euclidean coordinate system. """

    return X[:, :3] / X[:, [3]]

def exponential_map(xi_translation, xi_rotation):
    """ Converts a lie algebra entity to transformation matrix. """

    return lietorch.SE3.exp(torch.hstack([xi_translation, xi_rotation])).matrix()

def homogeneous_coordinates(X):
    """ Converts a set of points in Euclidean coordinate system to homogeneous coordinate system. """

    return torch.hstack([X, torch.ones((X.shape[0], 1))])

def inverse_transformation(T):
    """ Computes the inverse transformation of a 4x4 matrix. """

    rotation = T[:3, :3]
    translation = T[:3, [3]]
    return torch.vstack([torch.hstack([rotation.T, - rotation.T @ translation]), torch.Tensor([0, 0, 0, 1])])

def mean_squared_error(X_1, X_2):
    """ Computes the mean squared error between two point clouds. """

    return torch.mean(torch.sum((X_1 - X_2) ** 2, dim=1))

In [3]:
def getTransform():
    """ Generates a random 4x4 transformation matrix. """

    R = o3d.geometry.get_rotation_matrix_from_xyz(np.random.rand(3) * 2 * np.pi)
    T = np.random.rand(3) * 100
    transformation = np.vstack((np.hstack((R, T.reshape(-1, 1))), [0, 0, 0, 1]))
    return torch.Tensor(transformation)

In [4]:
def icp_alignment(
    T, max_iterations=750, tolerance=1e-2, lr_translation=1e-1, lr_rotation=1e-5, lr_point_cloud=1e-2
):
    """ Given a transformation matrix, transforms toothless.ply
    and recovers the transformation using Lie Group Optimization."""

    # Read the point cloud
    pcd = o3d.io.read_point_cloud("data/toothless.ply")
    X_P1 = torch.Tensor(np.asarray(pcd.points))

    # Homogeneous coordinate system
    X_P1 = homogeneous_coordinates(X_P1)

    # Apply the transformation
    X_P2 = apply_transformation(X_P1, T)

    # Add Gaussian noise to simulate practical scenarios
    X_P1 = add_gaussian_noise(X_P1)
    X_P2 = add_gaussian_noise(X_P2)

    # Predicted transformation and point cloud
    xi_translation = torch.zeros(3, requires_grad=True)
    xi_rotation = torch.zeros(3, requires_grad=True)
    X_P1.requires_grad_()

    # Setup optimizer for the lie group object
    optimizer = torch.optim.SGD([
        { 'params': xi_translation, 'lr': lr_translation },
        { 'params': xi_rotation, 'lr': lr_rotation },
        { 'params': X_P1, 'lr': lr_point_cloud }
    ], momentum=0.1)

    # Iterate until termination condition
    for iteration in range(max_iterations):

        # Apply the current transformation
        X_P2_pred = apply_transformation(X_P1, exponential_map(xi_translation, xi_rotation))

        # Compute the loss
        loss = mean_squared_error(X_P2, X_P2_pred)
        print(loss.item())

        # Optimization step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Check for convergence
        if loss < tolerance:
            break

    # Stop computing the gradients for the predicted transform
    xi_translation = xi_translation.detach()
    xi_rotation = xi_rotation.detach()
    X_P1 = X_P1.detach()

    # Apply the predicted transformation
    T_pred = exponential_map(xi_translation, xi_rotation)
    X_P2_aligned = apply_transformation(X_P2, inverse_transformation(T_pred))

    # Compute RMSE between the two transformed point clouds
    error = torch.sqrt(mean_squared_error(X_P1, X_P2_aligned))

    # Point clouds
    X = [ euclidean_coordinates(X_P1), euclidean_coordinates(X_P2_aligned) ]

    return T_pred, error, X

T = getTransform()
T_pred, alignment_error, X = icp_alignment(T)
T_error = torch.sqrt(torch.mean((T_pred - T) ** 2))

print('Original Transformation:\n', T)
print('Predicted Transformation:\n', T_pred)
print('Alignment Error:', alignment_error)
print('Error in Transformation:', T_error)

# Visualization
vis = o3d.visualization.Visualizer()
vis.create_window()
PC_1 = o3d.geometry.PointCloud()
PC_2 = o3d.geometry.PointCloud()
PC_1.points = o3d.utility.Vector3dVector(X[0].numpy())
PC_2.points = o3d.utility.Vector3dVector(X[1].numpy())
vis.add_geometry(PC_1)
vis.add_geometry(PC_2)
vis.poll_events()
vis.update_renderer()
vis.run()
vis.destroy_window()

17200.158203125
11272.6123046875
7426.12548828125
5227.2275390625
3908.1640625
3052.09326171875
2455.079345703125
2014.6085205078125
1675.831787109375
1407.35107421875
1190.0185546875
1011.4651489257812
863.2720947265625
739.4229125976562
635.4345703125
547.84375
473.89697265625
411.3642272949219
358.4073486328125
313.5006408691406
275.3679504394531
242.93812561035156
215.3106231689453
191.72869873046875
171.55458068847656
154.25245666503906
139.3714141845703
126.53240966796875
115.41722869873047
105.75846099853516
97.33170318603516
89.94820404052734
83.44993591308594
77.70421600341797
72.59942626953125
68.0418701171875
63.95280456542969
60.26609802246094
56.92595291137695
53.885414123535156
51.104820251464844
48.550777435302734
46.19498062133789
44.01336669921875
41.9855842590332
40.09425354003906
38.324581146240234
36.66395950317383
35.10139465332031
33.627620697021484
32.2344856262207
30.914995193481445
29.663028717041016
28.473262786865234
27.34099578857422
26.262083053588867
25.23

### 2. ICP-SLAM with LieTorch (10 marks)

For this part, you will be given a set of `N` transformation matrices (Each relative to position 0). Add a random amount of noise to each of the `N` sets of points. Estimate the Transforms as well as the original point cloud using LieTorch.

Visualize your results using open3d

In [5]:
def getTransforms(n):
    # Output shape - (n, 4, 4)

    T = []
    for _ in range(n):
        T.append(getTransform())

    return torch.stack(T)

In [6]:
def icp_slam(
    T, max_iterations=750, tolerance=1e-2, lr_translation=1e-1, lr_rotation=1e-5, lr_point_cloud=1e-4
):
    """ Given set of transformation matrices, transforms toothless.ply
    and recovers the transformation using Lie Group Optimization."""

    # Read the point cloud
    pcd = o3d.io.read_point_cloud("data/toothless.ply")
    X = torch.Tensor(np.asarray(pcd.points))

    # Dimensions
    num_points = X.shape[0]
    num_frames = T.shape[0]

    # Homogeneous coordinate system
    X = homogeneous_coordinates(X)

    # Apply the transformation
    X_frames = []
    for idx in range(num_frames):
        X_frames.append(apply_transformation(X, T[idx]))

    # Add Gaussian noise to simulate practical scenarios
    X = add_gaussian_noise(X)
    for idx in range(num_frames):
        X_frames[idx] = add_gaussian_noise(X_frames[idx])

    # Predict the transformation
    xi_translation = torch.zeros((num_frames, 3), requires_grad=True)
    xi_rotation = torch.zeros((num_frames, 3), requires_grad=True)
    X.requires_grad_()

    # Setup optimizer for the predicted transformation
    optimizer = torch.optim.SGD([
        { 'params': xi_translation, 'lr': lr_translation },
        { 'params': xi_rotation, 'lr': lr_rotation },
        { 'params': X, 'lr': lr_point_cloud }
    ], momentum=0.1)

    # Iterate until termination condition
    for iteration in range(max_iterations):

        # Apply the current transformation
        X_frames_pred = []
        for idx in range(num_frames):
            X_frames_pred.append(apply_transformation(X, exponential_map(xi_translation[idx], xi_rotation[idx])))

        # Compute the loss
        loss = 0
        for idx in range(num_frames):
            loss += mean_squared_error(X_frames[idx], X_frames_pred[idx])
        print(loss.item() / num_frames)

        # Optimization step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Check for convergence
        if loss < tolerance:
            break

    # Stop computing the gradients for the predictions
    xi_translation = xi_translation.detach()
    xi_rotation = xi_rotation.detach()
    X = X.detach()

    # Convert the predicted transformations to matrices
    T_pred = []
    for idx in range(num_frames):
        T_pred.append(exponential_map(xi_translation[idx], xi_rotation[idx]))

    # Apply the predicted transformation
    X = euclidean_coordinates(X)
    X_frames_pred = [ X ]
    for idx in range(num_frames):
        X_frames_pred.append(euclidean_coordinates( \
            apply_transformation(X_frames[idx], inverse_transformation(T_pred[idx]))))

    # Compute RMSE between the transformed point clouds
    error = 0
    for idx in range(num_frames):
        error += torch.sqrt(mean_squared_error(X, X_frames_pred[idx + 1])) / num_frames

    return T_pred, X_frames_pred, error

T = getTransforms(5)
T_pred, X, alignment_error = icp_slam(T)
T_error = 0
for idx in range(T.shape[0]):
    T_error += torch.sqrt(torch.mean((T[idx] - T_pred[idx]) ** 2)) / T.shape[0]
print('Alignment Error:', alignment_error)
print('Error in Transformation:', T_error)

# Visualization
vis = o3d.visualization.Visualizer()
vis.create_window()
PC = [ o3d.geometry.PointCloud() for _ in range(len(X)) ]
for idx in range(len(X)):
    PC[idx].points = o3d.utility.Vector3dVector(X[idx].numpy())
for idx in range(len(X)):
    vis.add_geometry(PC[idx])
vis.poll_events()
vis.update_renderer()
vis.run()
vis.destroy_window()

27984.584375
19060.4140625
13243.3125
10016.56953125
8191.09140625
7081.771875
6346.60546875
5816.1984375
5404.076171875
5064.33203125
4771.650390625
4511.58046875
4275.597265625
4058.505859375
3857.022265625
3668.97890625
3492.87265625
3327.5984375
3172.29609375
3026.2580078125
2888.8703125
2759.5859375
2637.9021484375
2523.3478515625
2415.4806640625
2313.8796875
2218.1453125
2127.898046875
2042.77890625
1962.446484375
1886.579296875
1814.8740234375
1747.0458984375
1682.8279296875
1621.97099609375
1564.24267578125
1509.4265625
1457.32197265625
1407.7423828125
1360.5154296875
1315.48173828125
1272.49384765625
1231.41640625
1192.12392578125
1154.50126953125
1118.44228515625
1083.84921875
1050.63212890625
1018.7080078125
988.0009765625
958.440625
929.9625
902.50703125
876.01923828125
850.4484375
825.748046875
801.874609375
778.788427734375
756.452001953125
734.8310546875
713.89375
693.610302734375
673.953125
654.896337890625
636.415869140625
618.4892578125
601.095166015625
584.2139160156